**We obtain our dataset by leveraging Reddit's PRAW API**

<!-- This is a better approach than using Push Shift because of this documented [error](https://github.com/iterative/aita_dataset/releases/tag/v.20.1.1). Push shift has a bug where recent posts are not properly queried. PRAW is slower but seems to be more accurate -->




In [ ]:
!pip install praw
import praw
import requests
import json
import pandas as pd
import time
import csv
# https://asyncpraw.readthedocs.io/en/stable/getting_started/quick_start.html for async praw
# for normal praw: https://praw.readthedocs.io/en/latest/getting_started/quick_start.html#read-only

In [ ]:
client_id = "REDACTED"
client_secret = "REDACTED"
username="REDACTED"
password="REDACTED"

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    password=password,
    user_agent="224n test script",
    username=username,
)

In [ ]:
print(reddit.read_only)
print(reddit.user.me())

In [ ]:
subreddit = reddit.subreddit("AmItheAsshole")
print(subreddit.display_name)
print(subreddit.title)
# print(subreddit.description)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/CS\ 224N/CS\ 224N\ Project
! ls # verify that you are in the right directory

In [ ]:
df = pd.read_csv("aita_clean.csv")
use_posts = df['id'].tolist()

In [ ]:
# print(use_posts)
top_comments = []
counter = 27485
f = open('comments.csv',"w")
writer = csv.writer(f, quoting=csv.QUOTE_ALL)
# 1/3 of doc: 32000
# 2/3 of doc: 60000
# 3/3 of doc: end

for idx in use_posts[counter:32000]:
  post = reddit.submission(idx)
  post.comments.replace_more(limit=4)
  flat_comments = post.comments.list()
  flat_comments.sort(key=lambda comment: comment.score, reverse=True)
  top_comment = flat_comments[0].body

  entry = [idx, top_comment]
  top_comments.append(entry)
  
  if counter % 1000 == 0:
    print(counter)
    d = {'id': [i[0] for i in top_comments], 'comments': [i[1] for i in top_comments] }
    df = pd.DataFrame(d)
    df.to_csv("/comments_batch_" +str(counter)+".csv", index=False) 
  counter += 1

print(top_comments)
df_comments = pd.DataFrame(top_comments, columns=['id','comments'])
df_comments.head()
f.close()

In [ ]:
post.comments.list()[1].body

In [ ]:
# f.close()
print(counter)
print(len(top_comments))
d = {'id': [i[0] for i in top_comments], 'comments': [i[1] for i in top_comments] }
df = pd.DataFrame(d)
df.to_csv("YOUR FILE PATH HERE" +str(counter)+".csv", index=False) 

32000
4515
